# 一、基本定义


在本实验中，我们用一个3×3的NumPy数组来表示八数码的状态。每个数字代表一个方块，0代表空格。

我们的目标是通过移动空格，使得初始状态经过一系列合法移动后，达到目标状态

## 定义启发函数

由于题目要求我们使用不同的启发函数进行搜索，给出搜索步数。所以应该先定义几种不同的启发函数：

0：曼哈顿距离：所有数字方块到其目标位置的“横纵坐标距离”之和。

1：错位数：当前状态中与目标状态不一致的数字方块的个数（不包括空格0）。

2：欧几里得距离：所有数字方块到其目标位置的欧几里得距离之和。

In [9]:
import numpy as np

def manhattan_distance(state, answer):
    distance = 0
    for num in range(1, 9):
        x1, y1 = np.where(state == num)
        x2, y2 = np.where(answer == num)
        distance += abs(x1[0] - x2[0]) + abs(y1[0] - y2[0])
    return distance

def misplaced_tiles(state, answer):
    return np.sum((state != answer) & (state != 0))

def euclidean_distance(state, answer):
    distance = 0
    for num in range(1, 9):
        x1, y1 = np.where(state == num)
        x2, y2 = np.where(answer == num)
        distance += np.sqrt((x1[0] - x2[0])**2 + (y1[0] - y2[0])**2)
    return distance

## 定义状态类

可以理解为一个board（二维数组，代表九宫格上的位置）就是一个状态

但是一个状态还需要其父状态，g（已走步数）等信息，因此需要定义成一个类

定义eq和hash是为了能够方便地用 np.array_equal 来判断两个board是否等价

In [10]:
class State:
    answer = np.array([[1, 2, 3], [8, 0, 4], [7, 6, 5]])
    def __init__(self, board, g=0, parent=None):
        self.board = board
        self.g = g  # 已走步数
        self.parent = parent
        self.zero_pos = tuple(np.argwhere(board == 0)[0])
    def __eq__(self, other):
        return np.array_equal(self.board, other.board)
    def __hash__(self):
        return hash(tuple(self.board.flatten()))

## 定义当前状态可到达的状态

In [11]:
def get_neighbors(state):
    neighbors = []
    x, y = state.zero_pos
    directions = [(-1,0),(1,0),(0,-1),(0,1)]
    for dx, dy in directions:
        nx, ny = x+dx, y+dy
        if 0 <= nx < 3 and 0 <= ny < 3:
            new_board = state.board.copy()
            # 调换位置，模拟方块的移动
            new_board[x, y], new_board[nx, ny] = new_board[nx, ny], new_board[x, y]
            neighbors.append(State(new_board, state.g+1, state))
    return neighbors

# 二、实现算法基本逻辑

In [12]:
import heapq
import itertools


def Astar(heuristic_id):
    heuristics = {
        0: manhattan_distance,
        1: misplaced_tiles,
        2: euclidean_distance
    }
    # 根据编号选择合适的启发函数，默认manhattan_distance
    heuristic_func = heuristics.get(heuristic_id, manhattan_distance)
    
    originState = State(np.array([[2, 7, 0], [8, 4, 1], [3, 5, 6]]))
    goal = State.answer
    
    open_list = []
    # 将open_list定义为优先队列（最小堆），优先队列会先比较第一个元素，第一个元素比完会比较第二个元素
    g = 0
    h = heuristic_func(originState.board, goal)
    f = g + h
    counter = itertools.count()  # 生成唯一自增序号
    heapq.heappush(open_list, (f, counter, originState))
    closed_set = set()
    step_count = 0
    
    while open_list:
        #  弹出最小值
        _, _, current = heapq.heappop(open_list)
        if np.array_equal(current.board, goal):
            # 回溯路径
            path = []
            node = current
            while node:
                path.append(node)
                node = node.parent
            path.reverse()
            print(f"搜索步数: {step_count}")
            print(f"最短路径步数: {len(path)-1}")
            # print("路径:")
            # for s in path:
            #     print(s.board)
            return
        
        step_count += 1
        closed_set.add(hash(current))
        for neighbor in get_neighbors(current):
            if hash(neighbor) in closed_set:
                continue
            f = neighbor.g + heuristic_func(neighbor.board, goal)
            heapq.heappush(open_list, (f,next(counter), neighbor))
    print("无解")

# 三、查看结果

曼哈顿距离：

In [13]:
Astar(0)

搜索步数: 4551
最短路径步数: 26


错位数：

In [14]:
Astar(1)

搜索步数: 54082
最短路径步数: 26


欧几里得距离：

In [15]:
Astar(2)

搜索步数: 7257
最短路径步数: 26
